In [129]:
import plotly
import json
import math
import re
import numpy as np
import plotly.graph_objs as go
from typing import List, Tuple, Dict, Any, Union
from queue import PriorityQueue

In [130]:
# read map_exp.txt
with open("map_exp.txt") as f:
    lines = f.readlines()
    # combine into one string
    map_exp = "".join(lines)
data = map_exp.split("data=")[1]
dim = int(math.sqrt(data.count("n=64")))
# print(dim)
# print(data)
matching = r'\{([0-9,.-]+)n'
# find all captures
matches = re.findall(matching, data)
map_arr = None
for match in matches:
  numbers = match.split(",")
  arr = []
  for number in numbers:
    if number != '':
      n = float(number)
      n = 0 if n ==0 else 1
      arr.append(n)
  if map_arr is None:
    map_arr = np.array(arr)
  else:
    map_arr = np.vstack((map_arr, arr))
map_arr = map_arr.reshape(dim, dim, 64)

In [131]:
# get the topmost block in each column
top_blocks = []
for i in range(dim):
  for j in range(dim):
    # get the highest value of k where map_arr[i][j][k] == 1
    k = 64
    while k > 0 and map_arr[i][j][k-1] == 0:
      k -= 1
    assert(map_arr[i][j][k] ==0)
    top_blocks.append((i, j, k))
print(top_blocks)

[(0, 0, 35), (0, 1, 34), (0, 2, 33), (0, 3, 32), (0, 4, 31), (0, 5, 30), (0, 6, 30), (0, 7, 29), (0, 8, 36), (1, 0, 36), (1, 1, 35), (1, 2, 34), (1, 3, 33), (1, 4, 31), (1, 5, 30), (1, 6, 30), (1, 7, 30), (1, 8, 36), (2, 0, 35), (2, 1, 34), (2, 2, 33), (2, 3, 32), (2, 4, 32), (2, 5, 32), (2, 6, 31), (2, 7, 31), (2, 8, 36), (3, 0, 34), (3, 1, 33), (3, 2, 32), (3, 3, 34), (3, 4, 32), (3, 5, 32), (3, 6, 32), (3, 7, 31), (3, 8, 35), (4, 0, 32), (4, 1, 31), (4, 2, 32), (4, 3, 34), (4, 4, 38), (4, 5, 38), (4, 6, 38), (4, 7, 32), (4, 8, 34), (5, 0, 31), (5, 1, 30), (5, 2, 32), (5, 3, 38), (5, 4, 38), (5, 5, 41), (5, 6, 38), (5, 7, 38), (5, 8, 33), (6, 0, 31), (6, 1, 31), (6, 2, 31), (6, 3, 38), (6, 4, 41), (6, 5, 42), (6, 6, 41), (6, 7, 39), (6, 8, 32), (7, 0, 31), (7, 1, 31), (7, 2, 32), (7, 3, 38), (7, 4, 38), (7, 5, 41), (7, 6, 38), (7, 7, 39), (7, 8, 31), (8, 0, 35), (8, 1, 34), (8, 2, 32), (8, 3, 32), (8, 4, 31), (8, 5, 30), (8, 6, 29), (8, 7, 29), (8, 8, 36)]


In [132]:
x = []
y = []
z = []
for i in range(dim):
  for j in range(dim):
    for k in range(64):
      if map_arr[i][j][k] == 1:
        x.append(i)
        y.append(j)
        z.append(k)

tx = []
ty = []
tz = []
for b in top_blocks:
  tx.append(b[0])
  ty.append(b[1])
  tz.append(b[2])
fig = go.Figure(data=[
    #  go.Scatter3d(x=x, y=y, z=z,
    #               mode='markers',
    #               marker=dict(size=2)
    #              ),
      go.Scatter3d(x=tx, y=ty, z=tz,
                    mode='markers',
                    marker=dict(size=10, color='red')
                    )
    ])


In [133]:
fig.show()

In [134]:
def manhattan_distance(a, b):
  return abs(a[0] - b[0]) + abs(a[1] - b[1]) + abs(a[2] - b[2])


def cost_matrix(top_blocks, map_arr):
  # create a cost matrix
  cost_matrix = np.full((len(top_blocks), len(top_blocks)), math.inf)
  for i in range(len(top_blocks)):
    for j in range(len(top_blocks)):
        # i and j must be neighbors, not including diagonals. they must not be the same block
        if i == j:
            continue
        if manhattan_distance(top_blocks[i], top_blocks[j]) == 1:
            cost_matrix[i][j] = 1
            continue
        # # check if the blocks have direct line of sight (same column, row) and no blocks in between
        # is_diff_x = top_blocks[i][0] != top_blocks[j][0] and (top_blocks[i][1] == top_blocks[j][1] and top_blocks[i][2] == top_blocks[j][2])
        # is_diff_y = top_blocks[i][1] != top_blocks[j][1] and (top_blocks[i][0] == top_blocks[j][0] and top_blocks[i][2] == top_blocks[j][2])
        # is_diff_z = top_blocks[i][2] != top_blocks[j][2] and (top_blocks[i][0] == top_blocks[j][0] and top_blocks[i][1] == top_blocks[j][1])
        # if is_diff_x:
        #   # check if there is a block in between
        #   for k in range(min(top_blocks[i][0], top_blocks[j][0]), max(top_blocks[i][0], top_blocks[j][0])):
        #     if map_arr[k][top_blocks[i][1]][top_blocks[i][2]] == 1 or map_arr[k][top_blocks[j][1]][top_blocks[j][2]] == 1:
        #       break
        #   if k == max(top_blocks[i][0], top_blocks[j][0]):
        #     # TODO: make the cost equal to distance
        #     cost_matrix[i][j] = abs(top_blocks[i][0] - top_blocks[j][0])
        # elif is_diff_y:
        #   # check if there is a block in between
        #   for k in range(min(top_blocks[i][1], top_blocks[j][1]), max(top_blocks[i][1], top_blocks[j][1])):
        #     if map_arr[top_blocks[i][0]][k][top_blocks[i][2]] == 1 or map_arr[top_blocks[j][0]][k][top_blocks[j][2]] == 1:
        #       break
        #   if k == max(top_blocks[i][1], top_blocks[j][1]):
        #     cost_matrix[i][j] = abs(top_blocks[i][1] - top_blocks[j][1])
        # elif is_diff_z:
        #   # check if there is a block in between
        #   for k in range(min(top_blocks[i][2], top_blocks[j][2]), max(top_blocks[i][2], top_blocks[j][2])):
        #     if map_arr[top_blocks[i][0]][top_blocks[i][1]][k] == 1 or map_arr[top_blocks[j][0]][top_blocks[j][1]][k] == 1:
        #       break
        #   if k == max(top_blocks[i][2], top_blocks[j][2]):
        #     cost_matrix[i][j] = abs(top_blocks[i][2] - top_blocks[j][2])
        # if cost_matrix[i][j] == math.inf:
        cost_matrix[i][j] = manhattan_distance(top_blocks[i], top_blocks[j])


  return cost_matrix

cost_mat = cost_matrix(top_blocks, map_arr)


In [135]:
def nearest_neighbor_tsp(top_blocks, map_arr, cost_mat):
  path = []
  visited = set()
  path.append((top_blocks[0], 0))
  visited.add(top_blocks[0])
  while len(visited) < len(top_blocks):
    # find the nearest neighbor
    nearest = None
    min_cost = math.inf
    for i in range(len(top_blocks)):
      if top_blocks[i] in visited:
        continue
      if cost_mat[path[-1][1]][i] < min_cost:
        min_cost = cost_mat[path[-1][1]][i]
        nearest = i
    path.append((top_blocks[nearest], nearest))
    visited.add(top_blocks[nearest])
  path = [p[0] for p in path]
  return path
path = nearest_neighbor_tsp(top_blocks, map_arr, cost_mat)
print(path)

[(0, 0, 35), (0, 1, 34), (0, 2, 33), (0, 3, 32), (0, 4, 31), (1, 4, 31), (1, 5, 30), (0, 5, 30), (0, 6, 30), (1, 6, 30), (1, 7, 30), (0, 7, 29), (2, 7, 31), (2, 6, 31), (2, 5, 32), (2, 4, 32), (2, 3, 32), (1, 3, 33), (1, 2, 34), (1, 1, 35), (1, 0, 36), (2, 0, 35), (2, 1, 34), (2, 2, 33), (3, 1, 33), (3, 0, 34), (3, 3, 34), (4, 3, 34), (4, 2, 32), (3, 2, 32), (3, 4, 32), (3, 5, 32), (3, 6, 32), (3, 7, 31), (4, 7, 32), (4, 8, 34), (3, 8, 35), (2, 8, 36), (1, 8, 36), (0, 8, 36), (4, 6, 38), (4, 5, 38), (4, 4, 38), (5, 4, 38), (5, 3, 38), (6, 3, 38), (7, 3, 38), (7, 4, 38), (7, 6, 38), (5, 6, 38), (5, 7, 38), (6, 7, 39), (7, 7, 39), (6, 6, 41), (5, 5, 41), (6, 4, 41), (6, 5, 42), (7, 5, 41), (8, 8, 36), (5, 8, 33), (6, 8, 32), (7, 8, 31), (8, 7, 29), (8, 6, 29), (8, 5, 30), (8, 4, 31), (8, 3, 32), (8, 2, 32), (7, 2, 32), (5, 2, 32), (6, 2, 31), (6, 1, 31), (6, 0, 31), (5, 0, 31), (4, 0, 32), (4, 1, 31), (5, 1, 30), (7, 1, 31), (7, 0, 31), (8, 0, 35), (8, 1, 34)]


In [136]:
def neighbors(p1, map_arr):
  dx = [-1, 1, 0, 0, 0, 0]
  dy = [0, 0, -1, 1, 0, 0]
  dz = [0, 0, 0, 0, -1, 1]
  neighbors = []
  for i in range(len(dx)):
    # check bounds
    if p1[0] + dx[i] < 0 or p1[0] + dx[i] >= len(map_arr):
      continue
    if p1[1] + dy[i] < 0 or p1[1] + dy[i] >= len(map_arr[0]):
      continue
    if p1[2] + dz[i] < 0 or p1[2] + dz[i] >= len(map_arr[0][0]):
      continue
    # check if the block is empty
    if map_arr[p1[0] + dx[i]][p1[1] + dy[i]][p1[2] + dz[i]] == 0:
      neighbors.append((p1[0] + dx[i], p1[1] + dy[i], p1[2] + dz[i]))
  return neighbors

def astar(p1, p2, map_arr):
    open_set = set()
    closed_set = set()
    open_set.add(p1)
    came_from = {}
    g_score = {}
    g_score[p1] = 0
    f_score = {}
    f_score[p1] = manhattan_distance(p1, p2)
    while len(open_set) > 0:
        current = None
        for p in open_set:
            if current is None or f_score[p] < f_score[current]:
                current = p
        if current == p2:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            return path[::-1]
        open_set.remove(current)
        closed_set.add(current)
        for p in neighbors(current, map_arr):
            if p in closed_set:
                continue
            if p not in open_set:
                open_set.add(p)
            tentative_g_score = g_score[current] + 1
            if p not in g_score or tentative_g_score < g_score[p]:
                came_from[p] = current
                g_score[p] = tentative_g_score
                f_score[p] = g_score[p] + manhattan_distance(p, p2)
# convert the path into discrete steps
new_path = []
for i in range(len(path) - 1):
    # use a star to get the steps from p1 to p2
    new_path += astar(path[i], path[i + 1], map_arr)
new_path.append(path[-1])
print(len(new_path), len(path))
path = new_path

177 81


In [137]:
px = []
py = []
pz = []
for point in path:
  px.append(point[0])
  py.append(point[1])
  pz.append(point[2])
# color by index so we can see the order
colors = []
for i in range(len(path)):
  colors.append(i)
fig = go.Figure(data=[
      go.Scatter3d(x=px, y=py, z=pz,
                    mode='lines', 
                    line=dict(color=colors),
                    hoverinfo='text',
                    hovertext=['(%d, %d, %d)' % (x[i], y[i], z[i]) for i in range(len(path))]
                    ),

    #  go.Scatter3d(x=x, y=y, z=z,
    #               mode='markers',
    #               marker=dict(size=2)
    #              ),
    ])
# fig.update_scenes(camera_projection_type='orthographic')
fig.show()

In [138]:
path

[(0, 1, 35),
 (0, 1, 34),
 (0, 2, 34),
 (0, 2, 33),
 (0, 3, 33),
 (0, 3, 32),
 (0, 4, 32),
 (0, 4, 31),
 (1, 4, 31),
 (1, 5, 31),
 (1, 5, 30),
 (0, 5, 30),
 (0, 6, 30),
 (1, 6, 30),
 (1, 7, 30),
 (0, 7, 30),
 (0, 7, 29),
 (0, 7, 30),
 (0, 7, 31),
 (1, 7, 31),
 (2, 7, 31),
 (2, 6, 31),
 (2, 6, 32),
 (2, 5, 32),
 (2, 4, 32),
 (2, 3, 32),
 (2, 3, 33),
 (1, 3, 33),
 (1, 3, 34),
 (1, 2, 34),
 (1, 2, 35),
 (1, 1, 35),
 (1, 1, 36),
 (1, 0, 36),
 (2, 0, 36),
 (2, 0, 35),
 (2, 1, 35),
 (2, 1, 34),
 (2, 2, 34),
 (2, 2, 33),
 (3, 2, 33),
 (3, 1, 33),
 (3, 1, 34),
 (3, 0, 34),
 (3, 1, 34),
 (3, 2, 34),
 (3, 3, 34),
 (4, 3, 34),
 (4, 2, 34),
 (4, 2, 33),
 (4, 2, 32),
 (3, 2, 32),
 (3, 2, 33),
 (2, 2, 33),
 (2, 3, 33),
 (2, 4, 33),
 (3, 4, 33),
 (3, 4, 32),
 (3, 5, 32),
 (3, 6, 32),
 (3, 7, 32),
 (3, 7, 31),
 (3, 7, 32),
 (4, 7, 32),
 (4, 7, 33),
 (4, 7, 34),
 (4, 8, 34),
 (4, 8, 35),
 (3, 8, 35),
 (3, 8, 36),
 (2, 8, 36),
 (1, 8, 36),
 (0, 8, 36),
 (1, 8, 36),
 (1, 8, 37),
 (2, 8, 37),
 (2, 8, 38),